In [ ]:
from datetime import datetime

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

from clouddrift import adapters, ragged

In [ ]:
def plot_trajectories(ds, linestyle):
    obs_ranges = np.cumsum(np.array([0, *ds["rowsize"]]))
    traj_ids = ds["id"].to_numpy().tolist()
    for traj_id in traj_ids:
        index = traj_ids.index(traj_id)
        start, end = obs_ranges[index], obs_ranges[index+1]
        selected = slice(start, end)
        
        plt.plot(
            ds["lon"].isel(obs=selected), ds["lat"].isel(obs=selected),
            linestyle=linestyle, ms=0.1,
            transform=ccrs.Geodetic(),
        )

# Plot the storms on the projection after applying the geodetic transformation
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

hurdat_ds = adapters.hurdat2.to_raggedarray().to_xarray()
gdp1h_ds = adapters.gdp1h.to_raggedarray(n_random_id=100).to_xarray()
criteria = dict(
    lat=(10, 50), 
    lon=(-80, -20), 
    time=(
        np.float64(datetime(2020, 8, 1).timestamp()), 
        np.float64(datetime(2020, 10, 1).timestamp())
    )
)

hurdat_ds = ragged.subset(hurdat_ds, criteria)
plot_trajectories(hurdat_ds, "-")